In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
from dateutil import parser as date_parser
import sys

class TimeseriesAgent:
    def __init__(self, csv_path):
        """Initialize the agent with a CSV file path."""
        self.csv_path = csv_path
        self.df = None
        self.date_column = None
        self.bond_identifiers = []  # Track unique bonds
        self.load_data()
    
    def load_data(self):
        """Load the CSV file and identify the date column."""
        try:
            self.df = pd.read_csv(self.csv_path)
            print(f"✓ Loaded {len(self.df)} rows from {self.csv_path}")
            
            # Auto-detect date column
            self.date_column = self._detect_date_column()
            if self.date_column:
                # Convert to datetime
                self.df[self.date_column] = pd.to_datetime(self.df[self.date_column])
                
                # Also convert maturity_date if it exists
                if 'maturity_date' in self.df.columns:
                    self.df['maturity_date'] = pd.to_datetime(self.df['maturity_date'])
                
                self.df = self.df.sort_values(self.date_column)
                print(f"✓ Detected date column: '{self.date_column}'")
                print(f"✓ Date range: {self.df[self.date_column].min().date()} to {self.df[self.date_column].max().date()}")
            else:
                print("⚠ Warning: Could not auto-detect date column")
            
            # Track bond identifiers
            if 'cusip' in self.df.columns:
                self.bond_identifiers = self.df['cusip'].unique().tolist()
                print(f"✓ Found {len(self.bond_identifiers)} unique bonds (CUSIPs)")
            
            if 'series' in self.df.columns:
                series_list = self.df['series'].unique().tolist()
                print(f"✓ Bond series: {', '.join(map(str, series_list))}")
            
            print(f"\nAvailable columns: {', '.join(self.df.columns.tolist())}")
            
        except FileNotFoundError:
            print(f"Error: File '{csv_path}' not found.")
            sys.exit(1)
        except Exception as e:
            print(f"Error loading CSV: {e}")
            sys.exit(1)
    
    def _detect_date_column(self):
        """Automatically detect which column contains dates."""
        date_keywords = ['date', 'time', 'timestamp', 'datetime', 'day']
        
        # First, check column names
        for col in self.df.columns:
            if any(keyword in col.lower() for keyword in date_keywords):
                return col
        
        # If not found by name, try parsing first row of each column
        for col in self.df.columns:
            try:
                pd.to_datetime(self.df[col].iloc[0])
                return col
            except:
                continue
        
        return None
    
    def parse_query(self, query):
        """Parse natural language query to extract date and intent."""
        query_lower = query.lower()
        
        # Determine query type
        intent = self._detect_intent(query_lower)
        
        # Extract date(s)
        dates = self._extract_dates(query)
        
        # Extract bond-specific filters
        cusip_filter = self._extract_cusip(query)
        series_filter = self._extract_series(query)
        tenor_filter = self._extract_tenor(query)
        
        return {
            'intent': intent,
            'dates': dates,
            'cusip': cusip_filter,
            'series': series_filter,
            'tenor': tenor_filter,
            'original_query': query
        }
    
    def _detect_intent(self, query_lower):
        """Detect what the user wants to do."""
        if any(word in query_lower for word in ['average', 'avg', 'mean']):
            return 'average'
        elif any(word in query_lower for word in ['sum', 'total']):
            return 'sum'
        elif any(word in query_lower for word in ['max', 'maximum', 'highest']):
            return 'max'
        elif any(word in query_lower for word in ['min', 'minimum', 'lowest']):
            return 'min'
        elif any(word in query_lower for word in ['range', 'between', 'from']):
            return 'range'
        elif any(word in query_lower for word in ['all bonds', 'all series', 'list']):
            return 'list'
        else:
            return 'single'  # Single date query
    
    def _extract_cusip(self, query):
        """Extract CUSIP from query if present."""
        # Look for CUSIP patterns (typically 9 characters alphanumeric)
        cusip_pattern = r'\b[A-Z0-9]{9}\b'
        matches = re.findall(cusip_pattern, query.upper())
        return matches[0] if matches else None
    
    def _extract_series(self, query):
        """Extract bond series from query."""
        query_upper = query.upper()
        if 'series' in self.df.columns:
            for series in self.df['series'].unique():
                if str(series).upper() in query_upper:
                    return series
        return None
    
    def _extract_tenor(self, query):
        """Extract tenor from query."""
        query_lower = query.lower()
        # Common tenor patterns: 10Y, 30Y, 2-year, etc.
        tenor_patterns = [r'\d+[yY]', r'\d+-year', r'\d+ year']
        for pattern in tenor_patterns:
            matches = re.findall(pattern, query)
            if matches:
                return matches[0]
        return None
    
    def _extract_dates(self, query):
        """Extract dates from natural language query."""
        dates = []
        
        # Handle relative dates
        today = datetime.now()
        
        if 'today' in query.lower():
            dates.append(today)
        elif 'yesterday' in query.lower():
            dates.append(today - timedelta(days=1))
        elif 'last week' in query.lower():
            dates.append(today - timedelta(weeks=1))
        elif 'last month' in query.lower():
            dates.append(today - timedelta(days=30))
        else:
            # Try to extract explicit dates using regex and dateutil
            # Common date patterns
            date_patterns = [
                r'\d{4}-\d{2}-\d{2}',  # YYYY-MM-DD
                r'\d{2}/\d{2}/\d{4}',  # MM/DD/YYYY
                r'\d{2}-\d{2}-\d{4}',  # DD-MM-YYYY
                r'[A-Za-z]+ \d{1,2},? \d{4}',  # Month DD, YYYY
                r'\d{1,2} [A-Za-z]+ \d{4}',  # DD Month YYYY
            ]
            
            for pattern in date_patterns:
                matches = re.findall(pattern, query)
                for match in matches:
                    try:
                        parsed_date = date_parser.parse(match)
                        dates.append(parsed_date)
                    except:
                        continue
        
        return dates if dates else [None]
    
    def query_data(self, query):
        """Execute the query and return results."""
        if self.date_column is None:
            return "Error: No date column detected. Please specify manually."
        
        parsed = self.parse_query(query)
        intent = parsed['intent']
        dates = parsed['dates']
        
        # Apply bond filters if specified
        filtered_df = self.df.copy()
        
        if parsed['cusip']:
            filtered_df = filtered_df[filtered_df['cusip'] == parsed['cusip']]
            if len(filtered_df) == 0:
                return f"No data found for CUSIP {parsed['cusip']}"
        
        if parsed['series']:
            filtered_df = filtered_df[filtered_df['series'] == parsed['series']]
            if len(filtered_df) == 0:
                return f"No data found for series {parsed['series']}"
        
        if parsed['tenor']:
            filtered_df = filtered_df[filtered_df['tenor'].str.contains(parsed['tenor'], case=False, na=False)]
            if len(filtered_df) == 0:
                return f"No data found for tenor {parsed['tenor']}"
        
        # Store filtered df for query methods
        self._temp_df = filtered_df
        
        if intent == 'list':
            return self._list_bonds(filtered_df)
        elif dates[0] is None and intent not in ['average', 'sum', 'max', 'min']:
            return "I couldn't extract a date from your query. Please try again with a specific date."
        
        # Execute based on intent
        if intent == 'single':
            return self._query_single_date(dates[0], filtered_df)
        elif intent == 'range':
            if len(dates) >= 2:
                return self._query_date_range(dates[0], dates[1], filtered_df)
            else:
                return "For range queries, please specify two dates."
        elif intent in ['average', 'sum', 'max', 'min']:
            return self._query_aggregation(dates, intent, filtered_df)
        
        return "Query type not recognized."
    
    def _query_single_date(self, target_date, filtered_df=None):
        """Query data for a specific date."""
        if filtered_df is None:
            filtered_df = self.df
        
        # Convert target_date to just date (no time) for comparison
        target_date_only = target_date.date()
        
        # Find exact match or nearest date
        filtered_df = filtered_df.copy()
        filtered_df['date_only'] = filtered_df[self.date_column].dt.date
        mask = filtered_df['date_only'] == target_date_only
        
        if mask.any():
            results = filtered_df[mask]
            output = f"\n📅 Data for {target_date_only} ({len(results)} bond(s)):\n"
            output += "=" * 80 + "\n\n"
            
            for idx, row in results.iterrows():
                output += f"Bond: {row.get('cusip', 'N/A')} | Series: {row.get('series', 'N/A')} | Tenor: {row.get('tenor', 'N/A')}\n"
                output += f"  Coupon: {row.get('coupon', 'N/A'):.3f}%\n"
                output += f"  Price: {row.get('price', 'N/A'):.4f}\n"
                output += f"  Yield: {row.get('yield', 'N/A'):.4f}%\n"
                output += f"  Maturity: {row.get('maturity_date', 'N/A')}\n"
                output += "-" * 80 + "\n"
            
            return output
        else:
            # Find nearest date
            filtered_df['date_diff'] = abs((filtered_df[self.date_column] - target_date).dt.total_seconds())
            nearest_idx = filtered_df['date_diff'].idxmin()
            nearest = filtered_df.loc[nearest_idx]
            nearest_date = nearest[self.date_column].date()
            
            output = f"\n⚠ No exact match for {target_date_only}\n"
            output += f"📅 Nearest date: {nearest_date}\n"
            output += "=" * 80 + "\n\n"
            output += f"Bond: {nearest.get('cusip', 'N/A')} | Series: {nearest.get('series', 'N/A')} | Tenor: {nearest.get('tenor', 'N/A')}\n"
            output += f"  Coupon: {nearest.get('coupon', 'N/A'):.3f}%\n"
            output += f"  Price: {nearest.get('price', 'N/A'):.4f}\n"
            output += f"  Yield: {nearest.get('yield', 'N/A'):.4f}%\n"
            output += f"  Maturity: {nearest.get('maturity_date', 'N/A')}\n"
            
            return output
    
    def _query_date_range(self, start_date, end_date, filtered_df=None):
        """Query data for a date range."""
        if filtered_df is None:
            filtered_df = self.df
        
        mask = (filtered_df[self.date_column] >= start_date) & (filtered_df[self.date_column] <= end_date)
        results = filtered_df[mask]
        
        if len(results) == 0:
            return f"No data found between {start_date.date()} and {end_date.date()}"
        
        output = f"\n📅 Data from {start_date.date()} to {end_date.date()} ({len(results)} records):\n"
        output += "=" * 80 + "\n\n"
        
        # Show summary statistics
        output += "Summary Statistics:\n"
        output += "-" * 80 + "\n"
        if 'price' in results.columns:
            output += f"  Average Price: {results['price'].mean():.4f}\n"
            output += f"  Min Price: {results['price'].min():.4f} | Max Price: {results['price'].max():.4f}\n"
        if 'yield' in results.columns:
            output += f"  Average Yield: {results['yield'].mean():.4f}%\n"
            output += f"  Min Yield: {results['yield'].min():.4f}% | Max Yield: {results['yield'].max():.4f}%\n"
        
        output += "\n" + "=" * 80 + "\n"
        output += "Detailed Records:\n"
        output += results[['date', 'cusip', 'series', 'coupon', 'price', 'yield', 'tenor']].to_string(index=False)
        
        return output
    
    def _query_aggregation(self, dates, agg_type, filtered_df=None):
        """Perform aggregation on data."""
        if filtered_df is None:
            filtered_df = self.df
        
        # For bond data, focus on price and yield
        output = f"\n📊 {agg_type.upper()} Statistics:\n"
        output += "=" * 80 + "\n\n"
        
        if 'price' in filtered_df.columns:
            if agg_type == 'average':
                value = filtered_df['price'].mean()
            elif agg_type == 'sum':
                value = filtered_df['price'].sum()
            elif agg_type == 'max':
                value = filtered_df['price'].max()
            elif agg_type == 'min':
                value = filtered_df['price'].min()
            output += f"Price {agg_type.title()}: {value:.4f}\n"
        
        if 'yield' in filtered_df.columns:
            if agg_type == 'average':
                value = filtered_df['yield'].mean()
            elif agg_type == 'sum':
                value = filtered_df['yield'].sum()
            elif agg_type == 'max':
                value = filtered_df['yield'].max()
            elif agg_type == 'min':
                value = filtered_df['yield'].min()
            output += f"Yield {agg_type.title()}: {value:.4f}%\n"
        
        if 'coupon' in filtered_df.columns:
            if agg_type == 'average':
                value = filtered_df['coupon'].mean()
            elif agg_type == 'sum':
                value = filtered_df['coupon'].sum()
            elif agg_type == 'max':
                value = filtered_df['coupon'].max()
            elif agg_type == 'min':
                value = filtered_df['coupon'].min()
            output += f"Coupon {agg_type.title()}: {value:.4f}%\n"
        
        output += f"\nTotal records: {len(filtered_df)}\n"
        
        return output
    
    def _list_bonds(self, filtered_df):
        """List all unique bonds in the dataset."""
        output = "\n📋 Bond Inventory:\n"
        output += "=" * 80 + "\n\n"
        
        if 'cusip' in filtered_df.columns:
            unique_bonds = filtered_df.groupby(['cusip', 'series', 'tenor', 'coupon']).size().reset_index(name='observations')
            output += f"Total unique bonds: {len(unique_bonds)}\n\n"
            output += unique_bonds.to_string(index=False)
        else:
            output += "No CUSIP information available.\n"
        
        return output
    
    def interactive_mode(self):
        """Start interactive query mode."""
        print("\n" + "=" * 80)
        print("🤖 BOND PRICE/YIELD DATABASE AGENT")
        print("=" * 80)
        print("\nAsk me questions about your bond data!")
        print("\nExample queries:")
        print("  • What was the price on 2024-12-15?")
        print("  • Show me yields for yesterday")
        print("  • What's the average yield?")
        print("  • Show data between 2024-01-01 and 2024-02-01")
        print("  • List all bonds")
        print("  • What was the yield for series A on 2024-12-01?")
        print("  • Show me 10Y bonds on 2024-11-15")
        print("\nType 'quit' or 'exit' to stop.\n")
        
        while True:
            try:
                query = input("❓ Your question: ").strip()
                
                if query.lower() in ['quit', 'exit', 'q']:
                    print("\n👋 Goodbye!")
                    break
                
                if not query:
                    continue
                
                result = self.query_data(query)
                print(result)
                print()
                
            except KeyboardInterrupt:
                print("\n\n👋 Goodbye!")
                break
            except Exception as e:
                print(f"❌ Error: {e}\n")


if __name__ == "__main__":
    # Usage example
    print("Timeseries Database Agent")
    print("-" * 60)
    
    csv_file = input("Enter the path to your CSV file: ").strip()
    
    agent = TimeseriesAgent(csv_file)
    agent.interactive_mode()

Timeseries Database Agent
------------------------------------------------------------
✓ Loaded 1540 rows from /Users/arifpras/Library/CloudStorage/OneDrive-Kemenkeu/01_Kemenkeu/TK4_202512/20251215_priceyield_cleaned.csv
✓ Detected date column: 'date'
✓ Date range: 2023-01-02 to 2025-12-12
✓ Found 6 unique bonds (CUSIPs)
✓ Bond series: FR96, FR95, FR101, FR100, FR104, FR103

Available columns: date, cusip, series, coupon, maturity_date, price, yield, tenor

🤖 BOND PRICE/YIELD DATABASE AGENT

Ask me questions about your bond data!

Example queries:
  • What was the price on 2024-12-15?
  • Show me yields for yesterday
  • What's the average yield?
  • Show data between 2024-01-01 and 2024-02-01
  • List all bonds
  • What was the yield for series A on 2024-12-01?
  • Show me 10Y bonds on 2024-11-15

Type 'quit' or 'exit' to stop.

I couldn't extract a date from your query. Please try again with a specific date.


⚠ No exact match for 2024-12-15
📅 Nearest date: 2024-12-12

Bond: ZI419444